In [ ]:
import numpy as np

from reflecto.simulator.simulator import ParamSet, XRRSimulator

qs = np.linspace(0.01, 0.3, 200)
sim = XRRSimulator(qs, n_layers=2, n_samples=1, has_noise=False, has_footprint=False)

thickness_true, rough_true, sld_true, R_true = next(sim.make_params_refl())

In [ ]:
import torch

q_torch = torch.tensor(qs, dtype=torch.float32).unsqueeze(1)
R_torch = torch.tensor(R_true, dtype=torch.float32).unsqueeze(1)

t_pred, sld_pred, sigma_pred = pinn_model(q_torch, R_torch)

In [ ]:
params_pred = []
for i in range(2):
    params_pred.append(
        ParamSet(
            thickness=t_pred[i].item(),
            roughness=sigma_pred[i].item(),
            sld=sld_pred[i].item()
        )
    )

R_pred_forward = sim.simulate_one(params_pred, has_noise=False, has_footprint=False)

error = np.mean((R_true - R_pred_forward)**2)
print("Forward-consistency error:", error)

In [ ]:
import matplotlib.pyplot as plt

plt.semilogy(qs, R_true, label="Original")
plt.semilogy(qs, R_pred_forward, label="PINN-forward")
plt.legend()
plt.show()